In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import resnet
from glob import glob

from siamese_network.data_pipeline.input_dataset import *
from siamese_network.model import embedding
from siamese_network.model import siamese_model

In [ ]:
# get list of paths to anchor and positive images
anchor_images = glob("../data/left/*")
positive_images = glob("../data/right/*")
len(anchor_images), len(positive_images)

# Plot random pairs of anchor and positive images

In [ ]:
n_pairs = 5
fig, axes = plt.subplots(n_pairs, 2, figsize=(8, 5*n_pairs))
fig.tight_layout(w_pad=1)

for i in range(n_pairs):
    ax_left, ax_right = axes[i]
    idx = np.random.randint(len(anchor_images))
    ax_left.imshow(load_image(anchor_images[idx]))
    ax_right.imshow(load_image(positive_images[idx]))

# Visualize triplets; anchor, positive and negative

In [ ]:
batch_size = 32
ds_train, ds_validation = create_triplet_dataset(anchor_images, positive_images, batch_size=batch_size)

In [ ]:
n_triplets = 5

batch_sample = list(ds_train.take(np.ceil(n_triplets / batch_size)).as_numpy_iterator())
batch_sample = np.concatenate(batch_sample, axis=1)

fig, axes = plt.subplots(n_triplets, 3, figsize=(12, 4.4*n_triplets))
fig.tight_layout(w_pad=1)

for i in range(n_triplets):
    ax_anchor, ax_pos, ax_neg = axes[i]
    ax_anchor.imshow(batch_sample[0, i])
    ax_pos.imshow(batch_sample[1, i])
    ax_neg.imshow(batch_sample[2, i])
axes[0, 0].set_title("Anchor", fontsize=16)
axes[0, 1].set_title("Positive", fontsize=16)
axes[0, 2].set_title("negative", fontsize=16);

# Checkout model inference

In [ ]:
for x in ds_train.take(1):
    break

In [ ]:
# creates embeddings for one batch of images
model_rn = embedding.create_embedding_model()

model_rn(x[0])

In [ ]:
# calculates distances for one batch of anchor, positive and negative triplets
model_siamese = siamese_model.create_siamese_model()

model_siamese(x)

In [ ]:
# implements the train and test step to use during training
model_training = siamese_model.SiameseModel()

model_training(x)

In [ ]:
# produces one loss value
model_training.test_step(x)